In [1]:
import glob
import random
import torch
import cv2
import torchvision.transforms as transforms
from torch.utils.data import Dataset
import matplotlib.pyplot as plt
import numpy as np
import torchvision
import torch.nn as nn
import torch.nn.functional as F

In [2]:
mean = [0.5071, 0.4867, 0.4408]
stdv = [0.2675, 0.2565, 0.2761]
transform = transforms.Compose([transforms.ToTensor(),transforms.Normalize(mean=mean, std=stdv)])

In [3]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 12, 3, 1, 1)
        self.pool = nn.MaxPool2d(2,2)
        self.conv2 = nn.Conv2d(12, 24, 3, 1, 1)
        self.drop = nn.Dropout2d(p=0.2)
        self.fc1 = nn.Linear(32*32*24, 120)
        self.fc2 = nn.Linear(120,84)
        self.fc3 = nn.Linear(84,6)
    
    def forward(self, x):
        x = F.relu(self.pool(self.conv1(x)))
        x = F.relu(self.pool(self.conv2(x)))
        x = F.dropout(self.drop(x))
        x = torch.flatten(x,1)
        x = x.view(-1, 32*32*24)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


In [35]:
net = Net()
net.load_state_dict(torch.load('/CS271P_Project/CNN_classification/cifar_net70.pth'))
net.eval()

Net(
  (conv1): Conv2d(3, 12, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(12, 24, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (drop): Dropout2d(p=0.2, inplace=False)
  (fc1): Linear(in_features=24576, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=6, bias=True)
)

In [36]:
image = cv2.imread('/CS271P_Project/result_image/1638297294.8872015.jpg')
image = transform(image).reshape(1,3,128,128)

In [37]:
outputs = net(image)
_, predicted = torch.max(outputs.data, 1)
print(predicted)

tensor([1])
